## Error Corrected At End Qubit Adder

In [1]:
%load_ext autoreload
%autoreload 2

import qiskit
import helpers.common as common
import helpers.gates as gates
import helpers.constants as constants

In [ ]:
# Create quantum registers
creq = []
areq = []
breq = []
for i in range(constants.bits):
    creq.append(common.logical_register(f"c{i}"))
    areq.append(common.logical_register(f"a{i}"))
    breq.append(common.logical_register(f"b{i}"))
breq.append(common.logical_register(f"b{constants.bits}"))

# Combine quantum registers into a single list
req_ce = []
for i in range(constants.bits):
    req_ce.extend(creq[i])
    req_ce.extend(areq[i])
    req_ce.extend(breq[i])
req_ce.extend(breq[constants.bits])



classical_req_ec = qiskit.ClassicalRegister(constants.bits + 1, "result")
qc = qiskit.QuantumCircuit(*req_ce, classical_req_ec)

# Add shor setup
for i in range(constants.bits):
    qc = gates.add_shor_setup(qc, creq[i])
    qc = gates.add_shor_setup(qc, areq[i])
    qc = gates.add_shor_setup(qc, breq[i])
qc = gates.add_shor_setup(qc, breq[constants.bits])

# Add X gates to set the initial values
for i in range(constants.bits):
    if constants.a[::-1][i] == "1":
        qc = gates.add_x(qc, areq[i])
    if constants.b[::-1][i] == "1":
        qc = gates.add_x(qc, breq[i])


# Add the carry and sum gates
for i in range(constants.bits):
    if i != constants.bits - 1:
        qc = gates.add_carry(qc, creq[i], areq[i], breq[i], creq[i + 1])
    else:
        qc = gates.add_carry(qc, creq[i], areq[i], breq[i], breq[i + 1])
        qc = gates.add_cx(qc, areq[i], breq[i])

for i in reversed(range(constants.bits)):
    qc = gates.add_sum(qc, creq[i], areq[i], breq[i])

    if i != 0:
        qc = gates.add_reverse_carry(qc, creq[i - 1], areq[i - 1], breq[i - 1], creq[i])

for i in range(constants.bits):
    qc = gates.add_shor_teardown(qc, creq[i])
    qc = gates.add_shor_teardown(qc, areq[i])
    qc = gates.add_shor_teardown(qc, breq[i])
qc = gates.add_shor_teardown(qc, breq[constants.bits])

print(f"{qc.num_qubits} qubits used")

for i in range(constants.bits + 1):
    qc.measure(breq[i][0], classical_req_ec[i])

# qc.draw("mpl")

117 qubits used


In [ ]:
raise common.StopExecution

simulator = common.SimulatedQuantumComputer(qc, 512)
counts = simulator.run()

print(f"Sum: {simulator.most_common()}")

StopExecution: 

In [ ]:
# raise common.StopExecution

simulator = common.RealQuantumComputer(qc, 512)
counts = simulator.run("result")

print(f"Backend: {simulator.backend_name()}")
print(f"Most likely state: {simulator.most_common()}")

simulator.plot()

Backend: ibm_brisbane
Most likely state: 00010
